# Using Selenium and Requests: _Hershel, Fjall Raven & Rains_

In this section we will be using selenium and beautiful soup to scrape all of the women's bags from Ahimasa Collective and Mimco.First, we will use selenium to get everylink for the bags, and then use Beautiful Soup to get the correct picture,price, product link, and description.

Some great sources used to help in this: 
- [Selenium Website for Locating Elements](https://selenium-python.readthedocs.io/locating-elements.html)
- [Stackover flow to Save Images](https://stackoverflow.com/questions/18497840/beautifulsoup-how-to-open-images-and-download-them)
- [Get all image src's with specific class](https://stackoverflow.com/questions/8289957/python-2-7-beautiful-soup-img-src-extract)

#### Import your needed Libraries:

In [5]:
import time
import requests
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import cv2
import numpy as np 
import pandas as pd
import re
from PIL import Image
import requests
from io import BytesIO

#### Create variables for each of the URL

We will do this for each of the pages we will be pulling from on _Hershel, Fjall Raven & Rains_.

In [9]:
hershel = ''
fjall_raven = 'https://www.fjallraven.us/collections/backpacks'
rains = 'https://www.us.rains.com/collections/backpacks'

### Rains:

We will start with Rains, grabbing all of their bags with every colors. The name of the picture files, the prices and the names of the items will be saved in a DataFrame.

In [96]:
page = requests.get(rains)
soup = BeautifulSoup(page.content, 'lxml')
class_1 = 'flexslider collection'
class_2 = 'details_outer'
divs_imgs = soup.findAll('div', {'class':class_1})

After inspecting, we can tell that these are all of the bags on the page. However, let's drop item 14 as it is not a bookbag. In the list comprehension we use [::2] to skip everyother item in the list of images for each item. These are the hover images which we do not need

In [97]:
images = [x.findAll('img')[::2] for x in divs_imgs]
del images[13]

In [98]:
flat_imgs = [img for img_li in images for img in img_li]

rains_colors = [x['alt'] for x in flat_imgs]
rains_urls = ['https:' + x['data-src'] for x in flat_imgs]

In [99]:
divs = soup.findAll('div', {'class':class_2})

In [100]:
name = [x.find('a')['title'] for x in divs]
pri = [x.find('a').find('span',{'class':'price_lg'}).text.replace('\n','').strip() for x in divs]
prices = [float(x.replace(' USD','').replace('$','')) for x in pri]

In [121]:
del name[13]
del prices[13]

In [111]:
div_num = [len(i) for i in images]

In [122]:
name_count = []
prices_count = []
for i in range(0,len(div_num)):
    name_count.append([name[i]]*div_num[i])
    prices_count.append([prices[i]]*div_num[i])

In [125]:
names_rains = [name for name_li in name_count for name in name_li]
prices_rains = [price for price_li in prices_count for price in price_li]

### Create a DataFrame and create a function to download pictures :

Below we will use the lists created above and make a dataframe. Then we will define a function to download the images into our directory, and save the image file name in a new column of the DataFrame.

In [133]:
table = [names_rains,prices_rains,rains_colors,rains_urls]
rains_df = pd.DataFrame(table).transpose()
rains_df.rename( columns={0:'Names', 1: 'Prices',2:'Color',3:'URL'},inplace=True)

In [156]:
rains_df.head(3)

,Names,Prices,Color,URL,img
0,Backpack,110,Black,https://cdn.shopify.com/s/files/1/2418/1767/pr...,rains__1.jpg
1,Backpack,110,Blue,https://cdn.shopify.com/s/files/1/2418/1767/pr...,rains__2.jpg
2,Backpack,110,Green,https://cdn.shopify.com/s/files/1/2418/1767/pr...,rains__3.jpg


In [20]:
def save_src_image_apply(company,short,url):
    start_time = time.time()
    time.sleep(1)
    
    try:
        global c
        c += 1
        response = requests.get(url)
        img = Image.open(BytesIO(response.content))
        image_save_loc = company+short+'_'+str(c)+'.jpg'
        img.save(image_save_loc, "JPEG")
        print("Saved "+short+"_{}".format(c)+".jpg")
        return short+"_"+ str(c)+".jpg"
    except Exception as e:
        return e

#### Save all images using an apply

In [153]:
c = 0
rains_df['img'] = rains_df.URL.apply(lambda x: save_src_image_apply('../../Rains_backpacks/','rains_',x))

Saved rains__1.jpg
Saved rains__2.jpg
Saved rains__3.jpg
Saved rains__4.jpg
Saved rains__5.jpg
Saved rains__6.jpg
Saved rains__7.jpg
Saved rains__8.jpg
Saved rains__9.jpg
Saved rains__10.jpg
Saved rains__11.jpg
Saved rains__12.jpg
Saved rains__13.jpg
Saved rains__14.jpg
Saved rains__15.jpg
Saved rains__16.jpg
Saved rains__17.jpg
Saved rains__18.jpg
Saved rains__19.jpg
Saved rains__20.jpg
Saved rains__21.jpg
Saved rains__22.jpg
Saved rains__23.jpg
Saved rains__24.jpg
Saved rains__25.jpg
Saved rains__26.jpg
Saved rains__27.jpg
Saved rains__28.jpg
Saved rains__29.jpg
Saved rains__30.jpg
Saved rains__31.jpg
Saved rains__32.jpg
Saved rains__33.jpg
Saved rains__34.jpg
Saved rains__35.jpg
Saved rains__36.jpg
Saved rains__37.jpg
Saved rains__38.jpg
Saved rains__39.jpg
Saved rains__40.jpg
Saved rains__41.jpg
Saved rains__42.jpg
Saved rains__43.jpg
Saved rains__44.jpg
Saved rains__45.jpg
Saved rains__46.jpg
Saved rains__47.jpg
Saved rains__48.jpg


In [155]:
rains_df.to_csv('../../rains_df.csv')

## Fjall Raven

Repeat earlier steps for Fjall Raven backpacks:

In [10]:
page_2 = requests.get(fjall_raven)
soup_2 = BeautifulSoup(page_2.content, 'lxml')
item_class = 'grid-view-item__link'
divs_items = soup_2.findAll('div', {'class':item_class})

Define variables to store the class names for each of the itmes that we need:

In [11]:
thumb_class ='product-single__thumbnails'
title_class = 'h4 grid-view-item__title'
price_class = 'h4 grid-view-item__price'

Use these classes in __list comprehensions__ to loop through the list of divs and further filter the items we need:

In [12]:
names_FR = [x.find('div', {'class':title_class}).text for x in divs_items]
prices_FR = [x.find('div',{'class':price_class}).text for x in divs_items]
links = ['https://www.fjallraven.us/'+x.find('a')['href'] for x in divs_items]
colors_divs = [x.find('select').findAll('option') for x in divs_items]
colors_FR = [[x.text.replace('\n','').strip() for x in i]for i in colors_divs]

Use each link to grab the images for each color bag:

In [13]:
del names_FR[20]
del prices_FR[20]
del links[20]

In [14]:
imgs_FR = []
skips = [2,2,1,1,2,2,1,1,1,1,1,1,1,1,1,1,1,4,3,3,1,1]
for i,link in enumerate(links):
    loop_page = requests.get(link)
    soup_loop = BeautifulSoup(loop_page.content, 'lxml')
    img_list = soup_loop.find('ul',{'class':thumb_class})
    img_items = img_list.findAll('a')[::skips[i]]
    imgs_FR.append(['https:'+x['data-zoom'] for x in img_items])

Use lengths of color and image lists for each item to make price and name lists long enough:

In [15]:
price_FR_count = []
names_FR_count = []
for i in range(0,len(names_FR)):
    names_FR_count.append([names_FR[i]]*len(imgs_FR[i]))
    price_FR_count.append([prices_FR[i]]*len(imgs_FR[i]))

Flatten all lists from list of lists to just a single list:

In [16]:
names_fjall = [name for name_li in names_FR_count for name in name_li]
prices_fjall = [price for price_li in price_FR_count for price in price_li]
colors_fjall = [color for color_li in colors_FR for color in color_li]
imgs_fjall = [img for imgs_li in imgs_FR for img in imgs_li]

In [17]:
table_FR = [names_fjall,prices_fjall,imgs_fjall]
fjallraven_df = pd.DataFrame(table_FR).transpose()
fjallraven_df.rename( columns={0:'Names', 1: 'Prices',2:'URL'},inplace=True)

Clean prices:

In [18]:
fjallraven_df.Prices = fjallraven_df.Prices.apply(lambda x: float(x.replace('$','')))


#### Save images and file names:

Use function from before to save images and create the column with the file name:

In [21]:
c = 0
fjallraven_df['img'] = fjallraven_df.URL.apply(lambda x: save_src_image_apply('../../fjall_raven_2/','FJ_',x))


Saved FJ__1.jpg
Saved FJ__2.jpg
Saved FJ__3.jpg
Saved FJ__4.jpg
Saved FJ__5.jpg
Saved FJ__6.jpg
Saved FJ__7.jpg
Saved FJ__8.jpg
Saved FJ__9.jpg
Saved FJ__10.jpg
Saved FJ__11.jpg
Saved FJ__12.jpg
Saved FJ__13.jpg
Saved FJ__14.jpg
Saved FJ__15.jpg
Saved FJ__16.jpg
Saved FJ__17.jpg
Saved FJ__18.jpg
Saved FJ__19.jpg
Saved FJ__20.jpg
Saved FJ__21.jpg
Saved FJ__22.jpg
Saved FJ__23.jpg
Saved FJ__24.jpg
Saved FJ__25.jpg
Saved FJ__26.jpg
Saved FJ__27.jpg
Saved FJ__28.jpg
Saved FJ__29.jpg
Saved FJ__30.jpg
Saved FJ__31.jpg
Saved FJ__32.jpg
Saved FJ__33.jpg
Saved FJ__34.jpg
Saved FJ__35.jpg
Saved FJ__36.jpg
Saved FJ__37.jpg
Saved FJ__38.jpg
Saved FJ__39.jpg
Saved FJ__40.jpg
Saved FJ__41.jpg
Saved FJ__42.jpg
Saved FJ__43.jpg
Saved FJ__44.jpg
Saved FJ__45.jpg
Saved FJ__46.jpg
Saved FJ__47.jpg
Saved FJ__48.jpg
Saved FJ__49.jpg
Saved FJ__50.jpg
Saved FJ__51.jpg
Saved FJ__52.jpg
Saved FJ__53.jpg
Saved FJ__54.jpg
Saved FJ__55.jpg
Saved FJ__56.jpg
Saved FJ__57.jpg
Saved FJ__58.jpg
Saved FJ__59.jpg
Saved 

In [22]:
bad_bags = ['FJ__17.jpg','FJ__18.jpg','FJ__29.jpg','FJ__53.jpg',
           'FJ__54.jpg','FJ__55.jpg','FJ__56.jpg','FJ__57.jpg','FJ__58.jpg',
           'FJ__59.jpg','FJ__60.jpg','FJ__61.jpg','FJ__62.jpg','FJ__63.jpg','FJ__64.jpg'
           'FJ__65.jpg','FJ__66.jpg','FJ__67.jpg','FJ__68.jpg','FJ__69.jpg',
           'FJ__70.jpg','FJ__78.jpg','FJ__83.jpg','FJ__84.jpg','FJ__85.jpg',
           'FJ__17.jpg','FJ__104.jpg','FJ__107.jpg','FJ__108.jpg','FJ__111.jpg',
           'FJ__112.jpg','FJ__114.jpg','FJ__115.jpg','FJ__117.jpg','FJ__118.jpg',
           'FJ__120.jpg','FJ__121.jpg','FJ__127.jpg','FJ__145.jpg','FJ__151.jpg',
            'FJ__152.jpg','FJ__153.jpg','FJ__157.jpg','FJ__17.jpg','FJ__17.jpg'
           'FJ__17.jpg','FJ__17.jpg','FJ__157.jpg','FJ__17.jpg','FJ__17.jpg','FJ__17.jpg',
           'FJ__17.jpg','FJ__17.jpg']

fjallraven_df = fjallraven_df[fjallraven_df.img.apply(lambda x: x in bad_bags)]
fjallraven_df.to_csv('../../fjallraven_df.csv')

#### Fjall Raven #2

In [23]:
fjallrave_2 = 'https://www.fjallraven.us/collections/kanken?page=1&sort_by=price-descending'
page_3 = requests.get(fjallrave_2)
soup_3 = BeautifulSoup(page_3.content, 'lxml')
item_class = 'grid-view-item__link'
divs_items_2 = soup_3.findAll('div', {'class':item_class})

For the most part our code to grab these images will not change. However, we will be adding a try-except. Some of the bags only have one picture and therefore do not have the same classes:

In [31]:
names_FR_2 = [x.find('div', {'class':title_class}).text for x in divs_items_2]
prices_FR_2 = [x.find('div',{'class':price_class}).text for x in divs_items_2]
links_2 = ['https://www.fjallraven.us/'+x.find('a')['href'] for x in divs_items_2]

In [43]:
imgs_FR_2 = []
single_photo = 'product-single__photos'
skips_2 = [10,10,1,4,1,3,1,2,2,1,2,1,2,1,1,1,1,2,1,2,1,1,2]
for i,link in enumerate(links_2):
    loop_page = requests.get(link)
    soup_loop = BeautifulSoup(loop_page.content, 'lxml')
    try:
        img_list = soup_loop.find('ul',{'class':thumb_class})
        img_items = img_list.findAll('a')[::skips_2[i]]
        imgs_FR_2.append(['https:'+x['data-zoom'] for x in img_items])
    except:
        img_list = soup_loop.find('div',{'class':single_photo})
        img_items = img_list.findAll('img')
        imgs_FR_2.append(['https:'+x['src'] for x in img_items])


In [44]:
price_FR_count_2 = []
names_FR_count_2 = []
for i in range(0,len(names_FR_2)):
    names_FR_count_2.append([names_FR_2[i]]*len(imgs_FR_2[i]))
    price_FR_count_2.append([prices_FR_2[i]]*len(imgs_FR_2[i]))

In [45]:
names_fjall_2 = [name for name_li in names_FR_count_2 for name in name_li]
prices_fjall_2 = [price for price_li in price_FR_count_2 for price in price_li]
imgs_fjall_2 = [img for imgs_li in imgs_FR_2 for img in imgs_li]

In [46]:
table_FR_2 = [names_fjall_2,prices_fjall_2,imgs_fjall_2]
fjallraven_df_2 = pd.DataFrame(table_FR_2).transpose()
fjallraven_df_2.rename( columns={0:'Names', 1: 'Prices',2:'URL'},inplace=True)

In [47]:
fjallraven_df_2.head()

,Names,Prices,URL
0,Kånken Messenger Bag,$240,https://cdn.shopify.com/s/files/1/0790/7429/pr...
1,Kånken Messenger Bag,$240,https://cdn.shopify.com/s/files/1/0790/7429/pr...
2,Kånken Messenger Bag,$240,https://cdn.shopify.com/s/files/1/0790/7429/pr...
3,Kånken Messenger Bag,$240,https://cdn.shopify.com/s/files/1/0790/7429/pr...
4,Kånken Tarpaulin Print Bag,$230,https://cdn.shopify.com/s/files/1/0790/7429/pr...


In [48]:
fjallraven_df_2.Prices = fjallraven_df_2.Prices.apply(lambda x: float(x.replace('$','')))

In [49]:
c = 166
fjallraven_df_2['img'] = fjallraven_df_2.URL.apply(lambda x: save_src_image_apply('../../fjall_raven_2/','FJ_',x))

Saved FJ__167.jpg
Saved FJ__168.jpg
Saved FJ__169.jpg
Saved FJ__170.jpg
Saved FJ__171.jpg
Saved FJ__172.jpg
Saved FJ__173.jpg
Saved FJ__174.jpg
Saved FJ__175.jpg
Saved FJ__176.jpg
Saved FJ__177.jpg
Saved FJ__178.jpg
Saved FJ__179.jpg
Saved FJ__180.jpg
Saved FJ__181.jpg
Saved FJ__182.jpg
Saved FJ__183.jpg
Saved FJ__184.jpg
Saved FJ__185.jpg
Saved FJ__186.jpg
Saved FJ__187.jpg
Saved FJ__188.jpg
Saved FJ__189.jpg
Saved FJ__190.jpg
Saved FJ__191.jpg
Saved FJ__192.jpg
Saved FJ__193.jpg
Saved FJ__194.jpg
Saved FJ__195.jpg
Saved FJ__196.jpg
Saved FJ__197.jpg
Saved FJ__198.jpg
Saved FJ__199.jpg
Saved FJ__200.jpg
Saved FJ__201.jpg
Saved FJ__202.jpg
Saved FJ__203.jpg
Saved FJ__204.jpg
Saved FJ__205.jpg
Saved FJ__206.jpg
Saved FJ__207.jpg
Saved FJ__208.jpg
Saved FJ__209.jpg
Saved FJ__210.jpg
Saved FJ__211.jpg
Saved FJ__212.jpg
Saved FJ__213.jpg
Saved FJ__214.jpg
Saved FJ__215.jpg
Saved FJ__216.jpg
Saved FJ__217.jpg
Saved FJ__218.jpg
Saved FJ__219.jpg
Saved FJ__220.jpg
Saved FJ__221.jpg
Saved FJ__

In [50]:
bad_bags_2 = ['FJ_177','FJ_178','FJ_179','FJ_180','FJ_181','FJ_182','FJ_183',
              'FJ_184','FJ_185','FJ_186','FJ_187','FJ_188','FJ_189','FJ_190',
              'FJ_191','FJ_192','FJ_193','FJ_194','FJ_195','FJ_196','FJ_197',
              'FJ_198','FJ_199','FJ_200','FJ_201','FJ_202','FJ_204','FJ_205',
              'FJ_206','FJ_207','FJ_208','FJ_209','FJ_210','FJ_211','FJ_234',
              'FJ_235','FJ_259','FJ_260','FJ_261','FJ_278','FJ_279','FJ_280',
              'FJ_296','FJ_297','FJ_298','FJ_314','FJ_315','FJ_316','FJ_317',
              'FJ_323','FJ_325','FJ_327','FJ_329','FJ_332','FJ_339','FJ_341',
              'FJ_343','FJ_348']

fjallraven_df_2 = fjallraven_df_2[fjallraven_df_2.img.apply(lambda x: x in bad_bags_2)]
fjallraven_df_2.to_csv('../../fjallraven_df_2.csv')